In [ ]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import csv
from io import StringIO

def chrome_page_data_command(vapp):
    javascript =  """
            function getElementXPath(elt) {
                var path = "";
                for (; elt && elt.nodeType == 1; elt = elt.parentNode) {
                    var idx = getElementIdx(elt);
                    var xname = elt.tagName;
                    if (idx > 1) xname += "[" + idx + "]";
                    path = "/" + xname + path;
                }
                return path;
            }

            function getElementIdx(elt) {
                var count = 1;
                for (var sib = elt.previousSibling; sib ; sib = sib.previousSibling) {
                    if (sib.nodeType == 1 && sib.tagName == elt.tagName) count++;
                }
                return count;
            }

            return getElementXPath(arguments[0]);
            """
    def page_elements(input_dict=None):
        driver = vapp['context']['driver']
        if not driver:
            print("WebDriver instance not found in context store.")
            return None
        
        # Switch to the specified tab
        tab_index = input_dict.get('tab_index', 0)
        try:
            driver.switch_to.window(driver.window_handles[tab_index])
        except IndexError:
            print(f"Error: Tab with index {tab_index} does not exist.")
            return None

        initial_element = driver.find_element(By.TAG_NAME, "body")
        # List to store data related to focused elements
        data = []
        full_data = []
        headers = ['ELEMENT_INDEX', 'TAG', 'TEXT', 'ID', 'NAME', 'INPUT_TYPE', 'VALUE', 'PLACEHOLDER']

        # Define XPath expressions for each type of element
        xpath_expressions = [
            "//input",
            "//a",
            "//select",
            "//option",
            "//button",
            "//form",
            "//textarea",
        ]
        x = 0
        for xpath_expr in xpath_expressions:
            elements = driver.find_elements(By.XPATH, xpath_expr)
            for i, focused_element in enumerate(elements):
                # Extract relevant information
                tag_name = focused_element.tag_name
                class_name = focused_element.get_attribute("class")
                input_value = focused_element.get_attribute("value")
                id = focused_element.get_attribute("id")
                placeholder = focused_element.get_attribute("placeholder")
                name = focused_element.get_attribute("name")
                text = focused_element.text if tag_name != "body" else None
                input_type = focused_element.get_attribute("type") if tag_name == "input" else None
                href = focused_element.get_attribute("href") if tag_name == "a" else None
                xpath = "XPATH: "+driver.execute_script(javascript, focused_element)
                
                obj = {
                    "ELEMENT_INDEX": x,
                    "TAG": tag_name,
                    "TEXT": text,
                    "ID": id,
                    "NAME": name,
                    "VALUE": input_value,
                    "PLACEHOLDER": placeholder,
                    "INPUT_TYPE": input_type,
                }
                
                data.append(obj)
                obj_copy = obj.copy()
                obj_copy['HREF'] = href
                obj_copy['XPATH'] = xpath
                full_data.append(obj_copy)
                x = x + 1

        csv_buffer = StringIO()
        writer = csv.DictWriter(csv_buffer, fieldnames=headers)
        writer.writeheader()
        for item in data:
            writer.writerow(item)

        # Get CSV data as a string
        csv_data = csv_buffer.getvalue()
        
        return {
            "csv_data" : csv_data,
            "full_data": full_data,
            "data": data
        }
    return page_elements

def chrome_click_command(vapp):
    """
    Input parameters:
        - input_dict['xpath']: XPath of the element to click
    Output parameters:
        - None
    """
    def click(input_dict=None):
        driver = vapp['context']['driver']
        xpath = input_dict['xpath']
        element = driver.find_element(By.XPATH, xpath)
        element.click()
        return {}
    return click

def chrome_sleep_command(vapp):
    """
    Input parameters:
        - input_dict['duration']: Duration to sleep in seconds
    Output parameters:
        - None
    """
    def sleep(input_dict=None):
        time.sleep(input_dict['duration'])
        return {}
    return sleep

def chrome_send_keys_command(vapp):
    """
    Input parameters:
        - input_dict['xpath']: XPath of the element to send keys
        - input_dict['keys']: Keys to send
    Output parameters:
        - None
    """
    keys_map = {}

    for key_name in dir(Keys):
        if key_name.isupper():  # Check if the attribute is an uppercase constant
            key_value = getattr(Keys, key_name)
            keys_map['key:'+key_name.lower()] = key_value
    
    def send_keys(input_dict=None):
        xpath = input_dict['xpath']
        keys_to_send = ""
        for key in input_dict['keys']:
            if key.startswith('key:'):  # Check if the argument starts with 'key:'
                key_name = key.split(':')[1]  # Extract the key name
                if key_name.lower() in keys_map:  # Check if the key name is in the keys_map
                    keys_to_send += keys_map[key_name.lower()]  # Append the corresponding Keys constant
            else:
                keys_to_send += key  # If not 'key:', append the argument as it is
        
        driver = vapp['context']['driver']
        element = driver.find_element(By.XPATH, xpath)
        element.send_keys(keys_to_send)
        return {}
    return send_keys

def chrome_open_url_command(vapp):
    """
    Input parameters:
        - input_dict['url']: URL to open
    Output parameters:
        - None
    """
    def open_url(input_dict=None):
        driver = vapp['context']['driver']
        driver.get(input_dict['url'])
        return {}
    return open_url



def chrome_start_command(vapp):
    def start(input_dict=None):
        vapp['context']['driver'] = webdriver.Chrome()
        return {}
    return start

def chrome_quit_command(vapp):
    def quit(input_dict=None):
        vapp['context']['driver'].quit()
        return {}
    return quit

def chrome_init(vapp):
    chrome_vapp['commands']['click'] = chrome_click_command(vapp)
    chrome_vapp['commands']['sleep'] = chrome_sleep_command(vapp)
    chrome_vapp['commands']['start'] = chrome_start_command(vapp)
    chrome_vapp['commands']['open_url'] = chrome_open_url_command(vapp)
    chrome_vapp['commands']['send_keys'] = chrome_send_keys_command(vapp)
    chrome_vapp['commands']['page_data'] = chrome_page_data_command(vapp)
    chrome_vapp['commands']['quit'] = chrome_quit_command(vapp)
    return



chrome_vapp = {
    "context" : {},
    "commands" : {},
    "init_script" : chrome_init
}

chrome_vapp['init_script'](chrome_vapp)

In [3]:
import os

# Commands
def userdata_append_command(vapp):
    """
    Input parameters:
        - input_dict['file_path']: File path
        - input_dict['content']: Content to append
        - input_dict['word_count'] (optional): Word count to update
    Output parameters:
        - status: 'success' or 'error'
        - message: Message indicating the result of the command
    """
    def append(input_dict=None):
        file_path = os.path.join(vapp['context']['root'], input_dict['file_path'])
        content = input_dict['content']
        try:
            with open(file_path, 'a') as file:
                file.write(content + '\n')
            # Update word count
            if input_dict is not None:
                word_count = len(content.split())
                input_dict['word_count'] = word_count
            return {'status': 'success', 'message': 'Appended to file'}
        except Exception as e:
            return {'status': 'error', 'message': str(e)}
    return append


def userdata_create_file_command(vapp):
    """
    Input parameters:
        - input_dict['file_path']: File path
    Output parameters:
        - status: 'success' or 'error'
        - message: Message indicating the result of the command
    """
    def create_file(input_dict=None):
        file_path = os.path.join(vapp['context']['root'], input_dict['file_path'])
        try:
            if not os.path.exists(file_path):
                with open(file_path, 'w'):
                    pass
                return {'status': 'success', 'message': 'File created'}
            else:
                return {'status': 'error', 'message': 'File already exists'}
        except Exception as e:
            return {'status': 'error', 'message': str(e)}
    return create_file

def userdata_get_content_command(vapp):
    """
    Input parameters:
        - input_dict['file_paths']: List of file paths
    Output parameters:
        - status: 'success' or 'error'
        - content: Content of the specified files concatenated together
        - message: Message indicating the result of the command
    """
    def get_content(input_dict=None):
        file_paths = input_dict['file_paths']
        if not file_paths:
            return {'status': 'error', 'message': 'No file paths provided'}
        
        try:
            content = ""
            for file_path in file_paths:
                full_path = os.path.join(vapp['context']['root'], file_path)
                if os.path.exists(full_path):
                    with open(full_path, 'r') as file:
                        content += file.read() + '\n'
                else:
                    return {'status': 'error', 'message': f'File not found: {file_path}'}
            return {'status': 'success', 'content': content}
        except Exception as e:
            return {'status': 'error', 'message': str(e)}
    return get_content


def userdata_list_keys_command(vapp):
    """
    Input parameters: None
    Output parameters:
        - status: 'success' or 'error'
        - keys: List of file names in the user data root folder
        - message: Message indicating the result of the command
    """
    def list_keys(input_dict=None):
        try:
            keys = []
            for root, _, files in os.walk(vapp['context']['root']):
                for file in files:
                    keys.append(file)
            return {'status': 'success', 'keys': keys}
        except Exception as e:
            return {'status': 'error', 'message': str(e)}
    return list_keys

def userdata_init(vapp):
    """
    Initialize the user data vApp.
    Creates a root folder for user data if it doesn't exist, and registers commands.
    """
    # Create a root folder if it doesn't exist
    root_folder = 'user_data'
    if not os.path.exists(root_folder):
        os.makedirs(root_folder)
    vapp['context']['root'] = root_folder
    
    # Register commands
    vapp['commands']['append'] = userdata_append_command(vapp)
    vapp['commands']['create_file'] = userdata_create_file_command(vapp)
    vapp['commands']['get_content'] = userdata_get_content_command(vapp)
    vapp['commands']['list_keys'] = userdata_list_keys_command(vapp)
    
    cmd.create("tree", Tree())
    return

# Initialization
user_data_vapp = {
    "context": {},
    "commands": {},
    "init_script": userdata_init
}



In [4]:
user_data_vapp['init_script'](user_data_vapp)

# Test create_file command
create_file_input = {'file_path': 'test.txt'}
create_file_result = user_data_vapp['commands']['create_file'](input_dict=create_file_input)
print("Create File Command Result:", create_file_result)

# Test append command
append_input = {'file_path': 'credentials.txt', 'content': 'Hello, world!'}
append_result = user_data_vapp['commands']['append'](input_dict=append_input)
print("Append Command Result:", append_result)

# Test get_content command
get_content_input = {'file_paths': ['test.txt', 'credentials.txt']}
get_content_result = user_data_vapp['commands']['get_content'](input_dict=get_content_input)
print("Get Content Command Result:", get_content_result)

# Test list_keys command
list_keys_result = user_data_vapp['commands']['list_keys']()
print("List Keys Command Result:", list_keys_result)

Create File Command Result: {'status': 'error', 'message': 'File already exists'}
Append Command Result: {'status': 'success', 'message': 'Appended to file'}
Get Content Command Result: {'status': 'success', 'content': '\nNew content for credentials file.Hello, world!\nHello, world!\nHello, world!\nHello, world!\n\n'}
List Keys Command Result: {'status': 'success', 'keys': ['credentials.txt', 'work_experience_skills.txt', 'test.txt', 'work_experience.txt']}


AttributeError: 'Token' object has no attribute 'NUMBER'